In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from pprint import pprint
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()
vram = torch.cuda.get_device_properties(0).total_memory / 1e9
print(f"Total VRAM: {vram} GB")

NVIDIA GeForce RTX 4080 SUPER
Total VRAM: 17.170956288 GB


In [3]:
# !pip install faiss-cpu # for vector db
# !pip install PyPDF2 # for parsing pdfs
# !pip install pandas
# !pip install scikit-learn  # For TfidfVectorizer, cosine distance
# !pip install langchain # for text splitting
# !pip install tqdm # for progress bar
# !pip install numpy

from typing import Dict, Any, List, Tuple
from helpers import (
    DocumentProcessor,
    VectorStore,
    HybridSearcher,
    Document,
    get_completion,
    get_embeddings_batch
)
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

2025-02-26 19:14:19,182 - INFO - Loading tokenizer and models...
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
2025-02-26 19:14:19,950 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
2025-02-26 19:14:26,498 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-02-26 19:14:33,078 - INFO - LLaMA 2 model loaded successfully.


In [ ]:
def load_documents(file_path):
    # Load and chunk document
    text = DocumentProcessor.load_pdf(file_path)
    documents = DocumentProcessor.chunk_text(text)
    
    # Create and return vector store
    vector_store = VectorStore()  # HybridSearcher()
    vector_store.create_index(documents, force_recreate=True)
    print(f"Processed {len(documents)} chunks")
    
    return vector_store

##UNCOMMENT THIS TO LOAD MULTIPLE DOCUMENTS##
# def load_documents(file_paths: List[str]) -> VectorStore:
#     """
#     Load and chunk multiple PDF documents, then create a combined vector store.

#     Args:
#         file_paths: List of file paths to PDF documents.

#     Returns:
#         vector_store: VectorStore containing indexed chunks from all documents.
#     """
#     all_documents = []

#     # Process each file
#     for file_path in file_paths:
#         print(f"Processing: {file_path}")
#         text = DocumentProcessor.load_pdf(file_path)
#         documents = DocumentProcessor.chunk_text(text)
#         all_documents.extend(documents)  # Combine all chunks

#     # Create and return vector store with all combined documents
#     vector_store = VectorStore()  # You can switch to HybridSearcher() if needed
#     vector_store.create_index(all_documents, force_recreate=True)
#     print(f"Processed {len(all_documents)} chunks from {len(file_paths)} documents")

    # return vector_store

# # List of file paths to process
# file_paths = [
#     "WF_benefits_book.pdf",
#     "another_document.pdf",
#     "more_documents.pdf"
# ]

# # Load and index multiple documents
# vector_store = load_documents(file_paths)

# For a single document, you can use:
vector_store = load_documents("WF_benefits_book.pdf")

Loading PDF pages:   0%|          | 0/458 [00:00<?, ?it/s]

Cleaning extracted text...


Processing chunks:   0%|          | 0/2074 [00:00<?, ?chunk/s]

Final number of chunks: 2074
Creating new index...


Generating embeddings:   0%|          | 0/2074 [00:00<?, ?it/s]

Saved index with 2074 documents
Processed 2074 chunks


In [5]:
query = "What is COBRA?"
results = vector_store.search(query, k=5)
results

print("Top 5 Search Results for:", query)
for idx, (doc, score) in enumerate(results, start=1):
    print(f"\nResult {idx}: (Similarity Score: {score:.4f})")
    print("-" * 50)
    # Display a snippet of the document content
    snippet = doc.content[:500]
    pprint(snippet)
    print("-" * 50)

Top 5 Search Results for: What is COBRA?

Result 1: (Similarity Score: 0.4822)
--------------------------------------------------
('E-4 If you have questions E-4 Keep your Plan informed of address changes E-5 '
 'Plan contact information E-5 Important information about payment E-5 COBRA '
 'administrative information E-5 General information E-5 Coverage and '
 'eligibility rights E-5 Primary qualifying events E-6 Your notification '
 'responsibilities — primary qualifying events E-6 Length of COBRA '
 'continuation coverage E-7 Extending COBRA continuation coverage E-8 Electing '
 'COBRA continuation coverage E-8 Summary of Benefits and corrections')
--------------------------------------------------

Result 2: (Similarity Score: 0.4628)
--------------------------------------------------
'Changing coverage What changes can you make during the year'
--------------------------------------------------

Result 3: (Similarity Score: 0.4591)
--------------------------------------------------

In [6]:
def query(question: str, vector_store, top_k: int = 5) -> str:
    """
    Process a question and return an answer with sources.

    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve

    Returns:
        Answer string
    """
    # Get relevant chunks
    relevant_chunks = vector_store.search(question, k=top_k)
    contents = "\n".join([x[0].content for x in relevant_chunks])

    prompt = f"""
    You are a helpful assistant specialized in extracting information from documents.

    I have the following question:
    "{question}"

    Here are the relevant document contents:
    {contents}

    Please provide a concise and accurate answer to the question based on the contents above.
    Only use the information present in the contents. If no relevant information is found, respond with "No".

    Answer:
    """

    # Call the completion function
    response = get_completion(prompt)
    print(response)

    # Clear GPU memory and force context termination
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    del prompt, contents, relevant_chunks  # Clear variables from memory

    return response  # Explicitly return the result to end the function



query("What is COBRA?", vector_store)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



    You are a helpful assistant specialized in extracting information from documents.

    I have the following question:
    "What is COBRA?"

    Here are the relevant document contents:
    E-4 If you have questions E-4 Keep your Plan informed of address changes E-5 Plan contact information E-5 Important information about payment E-5 COBRA administrative information E-5 General information E-5 Coverage and eligibility rights E-5 Primary qualifying events E-6 Your notification responsibilities — primary qualifying events E-6 Length of COBRA continuation coverage E-7 Extending COBRA continuation coverage E-8 Electing COBRA continuation coverage E-8 Summary of Benefits and corrections E-8 Cost of COBRA continuation coverage E-8 Payment of COBRA continuation coverage E-9 Changing COBRA continuation coverage E-11 Coordination of COBRA continuation coverage E-11 End of COBRA continuation coverage E-12 For more information E-12 Keep your Plan informed of address changes E-12 Plan contact 

'\n    You are a helpful assistant specialized in extracting information from documents.\n\n    I have the following question:\n    "What is COBRA?"\n\n    Here are the relevant document contents:\n    E-4 If you have questions E-4 Keep your Plan informed of address changes E-5 Plan contact information E-5 Important information about payment E-5 COBRA administrative information E-5 General information E-5 Coverage and eligibility rights E-5 Primary qualifying events E-6 Your notification responsibilities — primary qualifying events E-6 Length of COBRA continuation coverage E-7 Extending COBRA continuation coverage E-8 Electing COBRA continuation coverage E-8 Summary of Benefits and corrections E-8 Cost of COBRA continuation coverage E-8 Payment of COBRA continuation coverage E-9 Changing COBRA continuation coverage E-11 Coordination of COBRA continuation coverage E-11 End of COBRA continuation coverage E-12 For more information E-12 Keep your Plan informed of address changes E-12 Plan 

In [7]:
def get_relevant_chunks(
    question,
    vector_store,
    top_k = 3):
    """
    Retrieve relevant document chunks for a question
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of chunks to retrieve
    Returns:
        List of (document, relevance_score) tuples
    """
    return vector_store.search(question, k=top_k)

def generate_answer(
    question,
    relevant_chunks):
    """
    Generate an answer using retrieved chunks
    Args:
        question: User's question
        relevant_chunks: List of relevant documents and their scores
    Returns:
        Generated answer
    """
    # Format context from relevant chunks
    contexts = [
        f"[{i}] {doc.content}"
        for i, (doc, _) in enumerate(relevant_chunks, 1)
    ]
    
    prompt = f"""You are a helpful assistant.
    Answer the question based on the context. If the answer isn't in the context, say so.

    Question: {question}

    Context:
    {"/n/n".join(contexts)}

    Answer:"""
    
    return get_completion(prompt)

def query(
    question,
    vector_store,
    top_k = 10):
    """
    Process a question and return an answer with sources
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve
    Returns:
        Dictionary containing answer and sources
    """
    # Get relevant chunks
    relevant_chunks = get_relevant_chunks(question, vector_store, top_k)
    
    # Generate answer
    answer = generate_answer(question, relevant_chunks)
    
    # Format and return response
    return answer, relevant_chunks

question = "How does Wells Fargos benefits compare to Vanguard?"
answer, relevant_chunks = query(question, vector_store, top_k=10)

print("\nAnswer:", answer)

# Print sources
print("\nSources:")
for i, source in enumerate(relevant_chunks, 1):
    print(f"\n[{i}] Relevance score: {relevant_chunks[i-1][1]:.2f}")
    print(f"Preview: {relevant_chunks[i-1][0].content}")



Answer: You are a helpful assistant.
    Answer the question based on the context. If the answer isn't in the context, say so.

    Question: How does Wells Fargos benefits compare to Vanguard?

    Context:
    [1] For more information about employment classifications, refer to the “Who’s eligible to enroll” section on page 1-5 . Benefit plan options available to regular and fixed term employees Wells Fargo sponsors a number of benefit plans providing certain benefits to regular and fixed term employees. Some plans may offer more than one type of benefit option/n/n[2] In addition, the health savings account that may be set up separately is not a Wells Fargo-sponsored plan and is not subject to ERISA. For more information on the health savings accounts, refer to “Appendix C: Health Savings Accounts.” Employment classifications Employment classifications determine eligibility for most benefits programs, as well as rates for certain benefit plan options. Employment classifications in Wo